In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
from flatten_json import flatten
import folium
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import matplotlib.colors

In [2]:
location_data = pd.DataFrame({'lat':[0,40,1.5,25.6866142],'long':[0,-15,-8.5,-100.3161126],'Site':["Ecuador","Tula","Mexicali","Monterrey"]})
location_data.head()

,lat,long,Site
0,0.000000,0.000000,Ecuador
1,40.000000,-15.000000,Tula
2,1.500000,-8.500000,Mexicali
3,25.686614,-100.316113,Monterrey


In [3]:
df2 = pd.DataFrame()
for n_sites in location_data[["lat","long"]].to_numpy():
    print(n_sites,end='')
    lat = n_sites[0]
    long =n_sites[1]
    print("...done")
    url = "https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?&request=execute&identifier=SinglePoint&parameters=SI_EF_TILTED_SURFACE&userCommunity=SSE&tempAverage=CLIMATOLOGY&outputList=CSV&lat={}&lon={}".format(lat, long) 
    response = requests.get(url)
    solar_radiance=response.json()['features'][0]['properties']['parameter']
    newdf = pd.DataFrame.from_dict(solar_radiance,orient='index').reset_index()
    newdf['lat'] = lat
    newdf['long'] = long
    df2= df2.append(newdf,ignore_index=True)
print(n_sites,"...all done")

[0. 0.]...done
[ 40. -15.]...done
[ 1.5 -8.5]...done
[  25.6866142 -100.3161126]...done
[  25.6866142 -100.3161126] ...all done


In [4]:
df2.head()

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,lat,long
0,SI_EF_OPTIMAL,5.77,5.65,5.74,5.7,5.62,5.93,5.83,5.63,5.78,5.63,5.61,5.66,5.71,0.0,0.0
1,SI_EF_OPTIMAL_ANG,22.5,14,2.5,-10.5,-21,-26.5,-24,-15,-2.5,10,20,24.5,-0.5,0.0,0.0
2,SI_EF_OPTIMAL_ANG_ORT,S,S,S,N,N,N,N,N,N,S,S,S,N,0.0,0.0
3,SI_EF_TILTED_SURFACE_HORIZONTAL,5.41,5.52,5.73,5.63,5.33,5.43,5.42,5.48,5.78,5.56,5.34,5.25,5.49,0.0,0.0
4,SI_EF_TILTED_SURFACE_LATITUDE,5.41,5.52,5.73,5.63,5.33,5.43,5.42,5.48,5.78,5.56,5.34,5.25,5.49,0.0,0.0


In [80]:
df2 = pd.merge(df2,location_data,on=['lat','long'])
df2 = df2.rename(columns={"index":"Parameters"})
df2["avg_yearly_potential [kwh]"] = df2[list(range(0,13))].mean(axis=1)*365

In [6]:
transposed = df2.T
transposed.loc["avg_yearly_potential [kwh]"]=transposed.loc[list(range(0,13))].apply(pd.to_numeric,errors='ignore').mean(axis=0)*365
transposed
df2=transposed.T
df2.head()

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,lat,long,Site,avg_yearly_potential [kwh]
0,SI_EF_OPTIMAL,5.77,5.65,5.74,5.7,5.62,5.93,5.83,5.63,5.78,5.63,5.61,5.66,5.71,0,0,Ecuador,2084.99
1,SI_EF_OPTIMAL_ANG,22.5,14,2.5,-10.5,-21,-26.5,-24,-15,-2.5,10,20,24.5,-0.5,0,0,Ecuador,-182.5
2,SI_EF_OPTIMAL_ANG_ORT,S,S,S,N,N,N,N,N,N,S,S,S,N,0,0,Ecuador,NaN
3,SI_EF_TILTED_SURFACE_HORIZONTAL,5.41,5.52,5.73,5.63,5.33,5.43,5.42,5.48,5.78,5.56,5.34,5.25,5.49,0,0,Ecuador,2003.85
4,SI_EF_TILTED_SURFACE_LATITUDE,5.41,5.52,5.73,5.63,5.33,5.43,5.42,5.48,5.78,5.56,5.34,5.25,5.49,0,0,Ecuador,2003.85


In [137]:
default_lat = 31.7771
default_long = -40.24965
norm = Normalize(vmin=df2["avg_yearly_potential [kwh]"].loc[df2["Parameters"]=="SI_EF_OPTIMAL"].min(),vmax=df2["avg_yearly_potential [kwh]"].loc[df2["Parameters"]=="SI_EF_OPTIMAL"].max())
cmap=cm.Reds
m = cm.ScalarMappable(norm=norm, cmap=cmap)

world_map = folium.Map(location=[0,0],zoom_start=3)

for df_row in range(0,len(df3.values)):
    folium.CircleMarker(location=[df3["lat"].iloc[df_row],df3["long"].iloc[df_row]],radius=10,fill_opacity=0.5,
        popup=df3["Site"].iloc[df_row]+"\n"+str(df2["lat"].iloc[df_row])+","+str(df3["long"].iloc[df_row])+"\n"+str(round(df3["avg_yearly_potential [kwh]"].iloc[df_row],2))+"_kWh/year",
        fill_color=matplotlib.colors.to_hex(m.to_rgba(df3["avg_yearly_potential [kwh]"].iloc[df_row])),color=None).add_to(world_map)
world_map

In [24]:
df2["avg_yearly_potential [kwh]"].max()

12087.115384615383

In [77]:
norm = Normalize(vmin=0,vmax=1)
m = cm.ScalarMappable(norm=norm, cmap=cmap)

a=m.to_rgba(0.5)
matplotlib.colors.to_hex(a)

'#ff5c00'

In [131]:
df3 = df2[df2["Parameters"]=="SI_EF_OPTIMAL"]
df3["lat"].iloc[3]

KeyError: 3